In [5]:
import os
import rasterio
from rasterio.enums import Resampling

# Input folder containing the TIFF files
input_folder = r'/home/iwmi-wa/tmp_1tb/MOD17_NPP'
# Output folder where resampled and clipped TIFF files will be saved
output_folder = r'/home/iwmi-wa/tmp_1tb/MOD17_NPP_rs'
# Path to the mask layer (e.g., a shapefile or another raster file)
mask_layer = r"/efs/CWA/static_datasets/basin_mask/AFR_basinmask_1km.tif"

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# List all TIFF files in the input folder
tif_files = [f for f in os.listdir(input_folder) if f.endswith('.tif')]

# Open the mask layer to get its resolution and bounds
with rasterio.open(mask_layer) as mask_ds:
    target_resolution = mask_ds.res
    mask_bounds = mask_ds.bounds

# Iterate over each TIFF file for resampling and clipping
for tif_file in tif_files:
    input_tif = os.path.join(input_folder, tif_file)
    output_tif = os.path.join(output_folder, tif_file)

    with rasterio.open(input_tif) as src:
        # Create a window based on the mask layer's extent
        window = src.window(*mask_bounds)

        # Read data from the source raster within the window
        num_bands = 1  # Update with the correct number of bands for your raster
        data = src.read(window=window, out_shape=(num_bands, int(window.height), int(window.width)))

        # Calculate the scaling factors for resampling
        x_scale = target_resolution[0] / src.res[0]
        y_scale = target_resolution[1] / src.res[1]

        # Resample the data
        resampled_data = src.read(
            out_shape=(num_bands, int(data.shape[1] * y_scale), int(data.shape[2] * x_scale)),
            resampling=Resampling.nearest
        )

        # Update metadata for the output file and add compression options
        out_meta = src.meta.copy()
        out_meta.update({
            'driver': 'GTiff',
            'height': resampled_data.shape[1],
            'width': resampled_data.shape[2],
            'transform': src.window_transform(window),
            'compress': 'LZW',  # Replace 'LZW' with the desired compression method (e.g., 'JPEG', 'DEFLATE', 'PACKBITS', etc.)
            'predictor': 2  # Optional, set predictor for some compression methods
        })

        # Write the resampled and clipped dataset to a new file with compression
        with rasterio.open(output_tif, 'w', **out_meta) as dst:
            dst.write(resampled_data)

print("Resampling and clipping completed with compression.")


Resampling and clipping completed with compression.


In [11]:
import os
import rasterio
from rasterio.enums import Resampling

# Path to the folder containing the TIFF files to be resampled
input_folder = r'/home/iwmi-wa/tmp_1tb/MOD17_GPP_rs'

# Output folder for the resampled TIFF files
output_folder = r'/home/iwmi-wa/tmp_1tb/MOD17_GPP_c_rs'

# Target resolution (in degrees) for resampling in WGS 84
target_resolution = (0.009521932053583757535, 0.009521932053583757535)  # Change this to your desired resolution in degrees

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# List all TIFF files in the input folder
tif_files = [file for file in os.listdir(input_folder) if file.endswith('.tif')]

# Loop through the TIFF files in the input folder
for tif_file in tif_files:
    input_tiff_path = os.path.join(input_folder, tif_file)
    output_tiff_path = os.path.join(output_folder, tif_file)

    with rasterio.open(input_tiff_path) as src:
        # Calculate the new dimensions based on the target resolution
        width = int(src.width * src.transform.a / target_resolution[0])
        height = int(src.height * abs(src.transform.e) / target_resolution[1])

        # Resample the source TIFF file to the target resolution using nearest-neighbor method
        data = src.read(
            out_shape=(src.count, height, width),
            resampling=Resampling.nearest
        )

        # Update the profile with compression settings and the new resolution
        profile = src.profile
        profile.update(
            transform=rasterio.Affine(target_resolution[0], 0, src.transform.c, 0, -target_resolution[1], src.transform.f),
            width=width,
            height=height,
            compress='lzw'  # Set the compression method to LZW
        )

        # Write the resampled data to a new TIFF file with LZW compression
        with rasterio.open(output_tiff_path, 'w', **profile) as dst:
            dst.write(data)

print("Resampling complete. Resampled TIFF files saved in the output folder with nearest-neighbor resampling and LZW compression.")


Resampling complete. Resampled TIFF files saved in the output folder with nearest-neighbor resampling and LZW compression.
